[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_arima_forecast/EMQA_arima_forecast.ipynb)

# EMQA_arima_forecast

**ARIMA Forecast with Confidence Intervals**

Fit an ARIMA model to Brent crude oil and produce a 30-step-ahead forecast with 95% confidence intervals.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

brent = fetch('BZ=F')

# Fit ARIMA(1,1,1)
model = ARIMA(brent, order=(1, 1, 1))
result = model.fit()
print(result.summary())


In [ ]:
# Forecast 30 steps ahead
forecast_steps = 30
forecast = result.get_forecast(steps=forecast_steps)
fc_mean = forecast.predicted_mean
fc_ci = forecast.conf_int(alpha=0.05)

fig, ax = plt.subplots(figsize=(14, 6))

# Plot last 200 observations for context
hist_data = brent.iloc[-200:]
ax.plot(hist_data.index, hist_data.values, color=COLORS['blue'], linewidth=1.0,
        label='Historical')

# Forecast
ax.plot(fc_mean.index, fc_mean.values, color=COLORS['red'], linewidth=2.0,
        label='Forecast')

# Confidence interval
ax.fill_between(fc_ci.index, fc_ci.iloc[:, 0], fc_ci.iloc[:, 1],
                color=COLORS['red'], alpha=0.15, label='95% CI')

# Vertical line at forecast origin
ax.axvline(brent.index[-1], color=COLORS['gray'], linestyle='--', linewidth=0.8, alpha=0.7)

ax.set_title('ARIMA(1,1,1) Forecast — Brent Crude Oil', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Price (USD)')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10), frameon=False, ncol=3)

fig.tight_layout()
save_fig(fig, 'arima_forecast.pdf')
plt.show()
